In [2]:
import pandas as pd
import bw2data as bd
import bw2io as bi
import bw2analyzer as ba
import bw2calc as bc
import bw_processing as bwp
from pathlib import Path
import numpy as np
import csv
import sys

In [3]:
bd.projects.set_current('ei38-teaching-25')
ipcc=('IPCC 2013', 'climate change', 'GWP 100a')

In [28]:
ei = bd.Database('ei 3.8 cutoff')
activity= ei.get('6a7848cd768f5163834eec59c80de49b')
ei_pack=ei.datapackage()

In [5]:
#[act for act in ei if 'market for acrylic acid' in  act['name']][1].id

15498

In [26]:
heat_acts=[act for act in ei if 'heat production' in  act['name']
#and ('RER' in act['location'] or 'DE' in act['location'] or 'Europe without Switzerland' in act['location'])
#and 'heat' in act['reference product'] 
and 'heat pump' not in act['name']
and 'biomethane' not in act['name']
#and ('natural gas' in act['name'] or 'anthracite' in act['name'])
]

In [52]:
exchange =list(activity.technosphere())[0]
a= np.array((exchange.input.id, exchange.output.id), bwp.INDICES_DTYPE)

In [55]:
np.where(ei_pack.data[5] == a)

(array([157541], dtype=int64),)

In [32]:
index=[]
flow_list=[]
global depth
depth=0
max_depth=2

In [9]:
ei_pack.data[5][3]['row']

4428

In [ ]:
def recurse_supply(act, depth=0, max_depth=1, flow_dic={},amount=1):
    #global depth
    for exc in act.technosphere():
        
        if exc.input in heat_acts:
            amount = exc['amount']*amount
            flow_dic[exc.input]=amount
            
        elif depth< max_depth:
            depth +=1 
            depth, flow_dic = recurse_supply(exc.input,depth, max_depth, flow_dic, amount)
    
    depth -= 1
    return depth, flow_dic

In [ ]:
def recurse_supply(act, depth=0, max_depth=1, flow_dic={},amount=1):
    #global depth
    for exc in act.technosphere():
        
        if exc.input in heat_acts:
            amount = exc['amount']*amount
            flow_dic[exc.input]=amount
            
        elif depth< max_depth:
            depth +=1 
            depth, flow_dic = recurse_supply(exc.input,depth, max_depth, flow_dic, amount)
    
    depth -= 1
    return depth, flow_dic

In [61]:
def recurse_supply(act):
    global depth
    for exc in act.technosphere():
        if exc.input in heat_acts:
            flow_list.append(np.array((exc.input.id, exc.output.id), bwp.INDICES_DTYPE))
            print(exc.input)
        elif depth< max_depth:
            depth +=1 
            recurse_supply(exc.input)
    depth -= 1
        

In [62]:
flow_list=[]
global depth
depth=0
max_depth=3
recurse_supply(activity)

'heat production, natural gas, at boiler modulating >100kW' (megajoule, Europe without Switzerland, None)
'heat production, hardwood chips from forest, at furnace 50kW, state-of-the-art 2014' (megajoule, CH, None)
'heat production, hardwood chips from forest, at furnace 50kW' (megajoule, CH, None)
'heat production, light fuel oil, at boiler 100kW condensing, non-modulating' (megajoule, CH, None)
'heat production, light fuel oil, at boiler 100kW, non-modulating' (megajoule, CH, None)
'heat production, light fuel oil, at boiler 10kW condensing, non-modulating' (megajoule, CH, None)
'heat production, light fuel oil, at boiler 10kW, non-modulating' (megajoule, CH, None)
'heat production, mixed logs, at furnace 100kW, state-of-the-art 2014' (megajoule, CH, None)
'heat production, mixed logs, at furnace 100kW' (megajoule, CH, None)
'heat production, mixed logs, at furnace 30kW, state-of-the-art 2014' (megajoule, CH, None)
'heat production, mixed logs, at furnace 30kW' (megajoule, CH, None)
'

In [63]:
flow_list

[array((17601, 17526), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((16708, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((18519, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((6998, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((18590, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((23197, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((4916, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((15721, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((16902, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((9094, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((7050, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((13690, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((12052, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((13286, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((21713, 20823), dtype=[('row', '<i4'), ('col', '<i4')]),
 array((18899, 20823), dtype=[('row', '<i4')

In [74]:
ei.copy('electrified_ei')

Not able to determine geocollections for all datasets. This database is not ready for regionalization.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:44


Title: Writing activities to SQLite3 database:
  Started: 10/27/2022 14:16:40
  Finished: 10/27/2022 14:18:25
  Total time elapsed: 00:01:44
  CPU %: 31.80
  Memory %: 39.34


Brightway2 SQLiteBackend: electrified_ei

In [75]:
bd.databases

Databases dictionary with 4 object(s):
	biosphere3
	ei 3.8 cutoff
	electrified_ei
	pyrolysis_PE

In [65]:
ei_pack.data[6]

array([1.        , 0.07408187, 0.0727171 , ..., 1.        , 1.        ,
       1.        ], dtype=float32)

In [67]:
for exc in flow_list:
    print(ei_pack.data[6][np.where(ei_pack.data[5] == exc)[0]])

[9.956987]
[0.0596374]
[0.33027884]
[0.13457601]
[0.12569064]
[0.13457601]
[0.13457601]
[0.00010298]
[0.00092679]
[9.0708694e-05]
[0.00898016]
[0.00159112]
[0.01432008]
[8.0515114e-05]
[0.00072464]
[0.00015629]
[0.00140665]
[0.00269152]
[0.00403728]
[0.00099333]
[0.00022677]
[0.00022677]
[0.5211304]
[0.00951813]
[0.01584295]
[0.01608701]
[0.01594725]
[0.01633107]
[0.2692081]
[0.2533723]
[0.2402851]
[0.2371345]
[0.01584295]
[0.01608701]
[0.01594725]
[0.01633107]
[0.5211304]
[0.00951813]
[0.5211304]
[0.00951813]
[0.5211304]
[0.00951813]
[0.5211304]
[0.00951813]
[0.5211304]
[0.00951813]
[0.5211304]
[0.00951813]
[0.5211304]
[0.00951813]
[0.5211304]
[0.00951813]
[0.5211304]
[0.00951813]
[0.5211304]
[0.00951813]
[0.5211304]
[0.00951813]
[0.5211304]
[0.00951813]
[0.5211304]
[0.00951813]
[0.5211304]
[0.00951813]
[0.5211304]
[0.00951813]
[0.5211304]
[0.00951813]
[0.5211304]
[0.00951813]
[0.5211304]
[0.00951813]
